In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import joblib
import pretty_midi
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from MIDIComposingAI.get_back_data import melody_to_piano_roll, assembled_target_to_melody, assemblate_accompaniment_melody
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
from MIDIComposingAI.create_dataset import create_nparray_dataset, create_simple_dataset, extract_accompaniment_melody
import visual_midi
from os import listdir
from os.path import getsize

In [3]:
pm = joblib.load('../raw_data/pretty_midi/(Da Le) Yaleo')

In [4]:
acc, mel = create_simple_dataset(pm)

In [5]:
accs = []
for each in acc:
    accs.append(each)

In [6]:
acc.shape

(32, 128, 500)

In [7]:
i = 0
for each in accs:
    for peach in accs:
        if (each == peach).mean() < 1:
            i += 1
print(i)

950


In [8]:
acc.shape

(32, 128, 500)

In [9]:
bigger_examples_files = []
path = '../raw_data/pretty_midi'
directory = listdir(path)

In [10]:
# First we create a new dataset
bigger_examples_files = []
path = '../raw_data/pretty_midi'
directory = listdir(path)

for file in directory:
    if 200_000 < getsize(f'{path}/{file}') and getsize(f'{path}/{file}') < 300_000: # We don't want too big or too little files
        bigger_examples_files.append(joblib.load(f'{path}/{file}'))
    if len(bigger_examples_files) >= 10:
        break

directory = 'first_pres_dataset'
for i, file in enumerate(bigger_examples_files):
    create_nparray_dataset(file, directory, name=f'piece{i}')

In [3]:
X, y = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/piece0')
for i in range(1, 10):
    loaded = joblib.load(f'../raw_data/pandas_dataframes/first_pres_dataset/piece{1}')
    X = np.concatenate((X, loaded[0]))
    y = np.concatenate((y, loaded[1]))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [4]:
X_train = X_train.reshape((X_train.shape[0], -1))
X_test = X_test.reshape((X_test.shape[0], -1))

In [5]:
y_train = y_train.reshape(
    (
        y_train.shape[0], -1
    )
)

In [6]:
y_test = y_test.reshape(
    (
        y_test.shape[0], -1
    )
)

In [7]:
tree = DecisionTreeRegressor()
tree.fit(X_train, y_train)
joblib.dump(tree, 'tree.joblib')
predictions = tree.predict(X_test)

In [8]:
predicted_melodies = np.array([assembled_target_to_melody(prediction) for prediction in predictions])

In [9]:
true_melodies = np.array([assembled_target_to_melody(true_melody) for true_melody in y_test])

In [10]:
X_test = X_test.reshape(-1, 128, 500)

In [11]:
copy = X_test.copy()
copy_2 = X_test.copy()

In [12]:
assembled_predicted_music = np.array(
    [assemblate_accompaniment_melody(test, predicted_melody) for test, predicted_melody in zip(copy, predicted_melodies)],
)

assembled_true_music = np.array(
    [assemblate_accompaniment_melody(test, true_melody) for test, true_melody in zip(copy_2, true_melodies)],
)

In [13]:
for i, music in enumerate(assembled_predicted_music):
    try:
        piano_roll_to_pretty_midi(music, fs=50).write(f'../Musics/Predictions/pred{i}.mid')
    except:
        pass

In [14]:
for i, music in enumerate(assembled_true_music):
    try:
        piano_roll_to_pretty_midi(music, fs=50).write(f'../Musics/Actuals/true{i}.mid')
    except:
        pass

In [15]:
true_mel = piano_roll_to_pretty_midi(assembled_true_music[0], fs=50)
pred_mel = piano_roll_to_pretty_midi(assembled_predicted_music[0], fs=50)

In [17]:
test_mel = piano_roll_to_pretty_midi(assembled_target_to_melody(y_test[0]), fs=50)
test_acc = piano_roll_to_pretty_midi(X_test[0], fs=50)

In [18]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(test_acc)

Loading BokehJS ...

Column(id='1218', ...)

In [19]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(true_mel)

Loading BokehJS ...

Column(id='1486', ...)

In [20]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(pred_mel)

Loading BokehJS ...

Column(id='1761', ...)

In [53]:
# testing other files

In [54]:
bigger_examples_files = []
path = '../raw_data/pretty_midi'
directory = listdir(path)

for file in directory:
    if 200_000 > getsize(f'{path}/{file}'): # We don't want too big or too little files
        bigger_examples_files.append(joblib.load(f'{path}/{file}'))
    if len(bigger_examples_files) >= 5:
        break

In [31]:
directory = 'bigger_dataset'
for i, file in enumerate(bigger_examples_files):
    create_nparray_dataset(file, directory, name=f'test{i}')

In [32]:
X, y = joblib.load(f'../raw_data/pandas_dataframes/bigger_dataset/test0')
for i in range(1, 5):
    loaded = joblib.load(f'../raw_data/pandas_dataframes/bigger_dataset/test{1}')
    X = np.concatenate((X, loaded[0]))
    y = np.concatenate((y, loaded[1]))

In [33]:
X.shape

(46, 128, 500)

In [34]:
tree = joblib.load('tree.joblib')

In [10]:
X = X.reshape((X.shape[0], -1))
new_predictions = tree.predict(X)

In [11]:
predicted_melodies = np.array([assembled_target_to_melody(prediction) for prediction in new_predictions])
true_melodies = np.array([assembled_target_to_melody(true_melody) for true_melody in y])

In [12]:
X = X.reshape(-1, 128, 500)

In [13]:
assembled_predicted_music = np.array(
    [assemblate_accompaniment_melody(test, predicted_melody) for test, predicted_melody in zip(X, predicted_melodies)],
)

assembled_true_music = np.array(
    [assemblate_accompaniment_melody(test, true_melody) for test, true_melody in zip(X, true_melodies)],
)

In [14]:
for i, music in enumerate(assembled_predicted_music):
    try:
        piano_roll_to_pretty_midi(music, fs=50).write(f'../Musics/Predictions/test_pred{i}.mid')
    except:
        pass

for i, music in enumerate(assembled_true_music):
    try:
        piano_roll_to_pretty_midi(music, fs=50).write(f'../Musics/Actuals/test_true{i}.mid')
    except:
        pass    

In [15]:
true_mel = piano_roll_to_pretty_midi(assembled_true_music[1], fs=50)
pred_mel = piano_roll_to_pretty_midi(assembled_predicted_music[1], fs=50)

In [21]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(X[1]))

Loading BokehJS ...

Column(id='1789', ...)

In [16]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(true_mel)

Loading BokehJS ...

Column(id='1166', ...)

In [17]:
%%capture --no-display

coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(pred_mel)

Loading BokehJS ...

Column(id='1376', ...)